## Imports

In [1]:
# Super Mario Bros env dependencies
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, RIGHT_ONLY
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack, GrayScaleObservation, TransformObservation

# Torch
import torch
import torch.utils
import torch.nn as nn

# Networks to Evaluate
import sys, os
def add_to_path(model_dir):
    notebook_file = os.path.dirname("CNN_Feature_Visualisation.ipynb")
    path2add = os.path.normpath(os.path.abspath(os.path.join(notebook_file, os.path.pardir, model_dir)))
    if (not (path2add in sys.path)):
        print(f'updating path to include: {path2add}')
        sys.path.append(path2add)
add_to_path('DQN')
from OldAgent import MarioNet, Mario
add_to_path('a2c')
from a2c.model import ACNetwork

# CNN Visualisation (Lucent)
from lucent.optvis import render, param, transform, objectives
from lucent.misc.io import show
from lucent.modelzoo.util import get_model_layers

# Utilities
from utils.wrappers import ResizeObservation, SkipFrame
import utils.helper
from utils.config import Config
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from pathlib import Path
import torch.utils
import torch._utils

/home/whiffingj/bin/anaconda3/envs/rl_vis/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


updating path to include: /home/whiffingj/dev/uni/CM50270_CW2/src/DQN
updating path to include: /home/whiffingj/dev/uni/CM50270_CW2/src/a2c


## Model and Env Prep

In [2]:
def create_env(random = False, movement = SIMPLE_MOVEMENT):
    env_name = "SuperMarioBros"
    if random:
        env_name += "RandomStage"
    env_name += '-v3'
    env = gym_super_mario_bros.make(env_name)
    env = JoypadSpace(env, movement)
    return env, env_name

def run_model(env, steps, get_action, get_obs, image_dir):
    state = env.reset()
    frame = 0    
    for step in range(steps):
        obs = get_obs(state)
        if len(obs.shape) == 3 and obs.shape[0] == 4:
                obs = np.concatenate(obs, axis=0) # if we have 4 inputs, we want them organised left to right
        Image.fromarray(obs).save(os.path.join(image_dir, f"frame_{frame}.png"))
        frame += 1

        action = get_action(state)
        next_state, reward, done, _ = env.step(action)
        state = next_state
        
        if done:
            state = env.reset()

print(f"Is cuda supported on the system? {'Yes' if torch.cuda.is_available() else 'No'}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Is cuda supported on the system? Yes


## Setup Feature Visualisation Functions

In [3]:
# Code derived from: https://colab.research.google.com/github/greentfrapp/lucent-notebooks/blob/master/notebooks/feature_inversion.ipynb#scrollTo=d47pkOPKvNjs
@objectives.wrap_objective()
def dot_compare(layer, batch=1, cossim_pow=0):
    def inner(T):
        dot = (T(layer)[batch] * T(layer)[0]).sum()
        mag = torch.sqrt(torch.sum(T(layer)[0]**2))
        cossim = dot/(1e-6 + mag)
        return -dot * cossim ** cossim_pow
    return inner

transforms = [
    transform.pad(8, mode='constant', constant_value=.5),
    transform.jitter(8),
    transform.random_scale([0.9, 0.95, 1.05, 1.1] + [1]*4),
    transform.random_rotate(list(range(-5, 5)) + [0]*5),
    transform.jitter(2),
]

def get_param_f(img, device):
    img = torch.tensor(img).to(device)
    # Initialize parameterized input and stack with target image
    # to be accessed in the objective function
    params, image_f = param.image(img.shape[1], channels=img.shape[0])
    def stacked_param_f():
        return params, lambda: torch.stack([image_f()[0], img])

    return stacked_param_f

def feature_inversion(img, device, layer, model, n_steps=512, cossim_pow=0.0):  
    obj = objectives.Objective.sum([
        1.0 * dot_compare(layer, cossim_pow=cossim_pow),
        objectives.blur_input_each_step(),
    ])

    param_f = get_param_f(img, device)
    images = render.render_vis(model, obj, param_f, transforms=transforms, preprocess=False, thresholds=(n_steps,), show_image=False, progress=False)
    return images

def add_image_to_figure(ax, image, description, fig_args=dict()):
    ax.imshow(image, **fig_args) # Input frames
    ax.yaxis.set_visible(False)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
    ax.set_xlabel(description)

def visualise_cnn_layer_neurons(src_img_path, model, layers, device, convergence_steps, desc, out_img_path):
    if src_img_path != "NOISE":
        stacked_image = np.array(Image.open(src_img_path), np.float32)
        image = stacked_image.reshape((4, 84, 84))
    else:
        stacked_image = np.random.uniform(0, 255, (4, 84, 84))
        image = stacked_image

    processed_layers = []
    for layer_id, neurons, layer_idx in layers:
        print(f"Processing Layer-{layer_idx}")
        channel_activation = feature_inversion(image, device, layer_id, model, n_steps=convergence_steps)[0]

        neuron_activations = []
        param_f = get_param_f(image, device)
        for neuron in range(neurons):
            obj = f"{layer_id}:{neuron}"
            # print(f"Generating Activation Image For Neuron-{neuron}")
            neuron_activations.append(render.render_vis(model, obj, param_f, transforms=transforms, preprocess=False, thresholds=(convergence_steps,), show_image=False, progress=False)[0])

        processed_layers.append((channel_activation, neuron_activations))

    max_val = -np.inf
    min_val = np.inf
    activation_images = []
    for channel_act, neurons_acts in processed_layers:
        # get activation image
        processed_channel_act = channel_act[0].transpose([2, 0, 1])
        current_max = np.max(processed_channel_act)
        current_min = np.min(processed_channel_act)

        processed_neurons_acts = []
        for neurons_act in neurons_acts:
            processed_neurons_act = neurons_act[0].transpose([2, 0, 1])
            processed_neurons_acts.append(processed_neurons_act)
            current_n_max = np.max(processed_neurons_act)
            current_n_min = np.min(processed_neurons_act)
            if current_n_max > max_val:
                max_val = current_n_max
            if current_n_min < current_min:
                current_min = current_n_min

        if current_max > current_max:
            current_max = current_max
        if current_min < min_val:
            min_val = current_min

        activation_images.append((processed_channel_act, processed_neurons_acts))

    plt.rcParams['figure.dpi'] = 200
    fig = plt.figure(constrained_layout=True, figsize=(6, 8))
    total_cols = 5
    total_rows = len(layers)+1#int(np.sum([l[1] for l in layers]) / total_cols) + len(layers) + 1
    gs = fig.add_gridspec(ncols=total_cols, nrows=total_rows)
    input_ax = fig.add_subplot(gs[0,:])
    add_image_to_figure(ax=input_ax, image=np.concatenate(image, axis=1), description="Input Frame Stack (Left -> Right: Frames 0-3)", fig_args=dict(cmap='gray', vmin=0, vmax=255))
    for _, neurons, layer_idx in layers:
        print(f"Processing Activation Image For Layer-{layer_idx}, {neurons}")
        channel_activation, neuron_activations = activation_images[layer_idx]
        channel_rgba_ax = fig.add_subplot(gs[layer_idx+1,0])
        channel_stacked_ax = fig.add_subplot(gs[layer_idx+1,1:])
        add_image_to_figure(ax=channel_rgba_ax, image=channel_activation.transpose([1, 2, 0]), description=f"ConvLayer-{layer_idx}-RGBA")
        add_image_to_figure(ax=channel_stacked_ax, image=np.concatenate(channel_activation, axis=1), description=f"ConvLayer-{layer_idx}-Stacked", fig_args=dict(cmap='plasma', vmin=min_val, vmax=max_val))
        
        neurons_fig = plt.figure(figsize=(18, 14))
        neurons_total_cols = 8
        neurons_total_rows = int(neurons / neurons_total_cols)
        neurons_gs = fig.add_gridspec(ncols=neurons_total_cols, nrows=neurons_total_rows)
        for neuron_x in range(neurons_total_cols):
            for neuron_y in range(neurons_total_rows):
                neuron_activation = neuron_activations[(neuron_x*neurons_total_rows) + neuron_y]
                neuron_rgba_ax = neurons_fig.add_subplot(neurons_gs[neuron_y,neuron_x])
                add_image_to_figure(ax=neuron_rgba_ax, image=neuron_activation.transpose([1, 2, 0]), description=f"Neuron-{neuron_x + (neuron_y*neurons_total_rows)}-RGBA")
        neurons_fig.suptitle(f"RGBA Neurons For Layer {layer_idx} - {desc}")
        neurons_fig.tight_layout()
        # neurons_fig.show()
        neurons_fig.savefig(f"{out_img_path}_layer-{layer_idx}-neurons-rgba.png")
        neurons_fig.clf()

        neurons_fig = plt.figure(figsize=(18, 14))
        neurons_total_cols = 16
        neurons_total_rows = int(neurons / 4)
        neurons_gs = fig.add_gridspec(ncols=neurons_total_cols, nrows=neurons_total_rows)
        for neuron_x in range(4):
            for neuron_y in range(neurons_total_rows):
                neuron_activation = neuron_activations[(neuron_x*neurons_total_rows) + neuron_y]
                neuron_stacked_ax = neurons_fig.add_subplot(neurons_gs[neuron_y,(neuron_x*4):(neuron_x*4)+4])
                add_image_to_figure(ax=neuron_stacked_ax, image=np.concatenate(neuron_activation, axis=1), description=f"Neuron-{neuron_x + (neuron_y*neurons_total_rows)}-Stacked", fig_args=dict(cmap='plasma', vmin=min_val, vmax=max_val))
        neurons_fig.suptitle(f"Neurons For Layer {layer_idx} - {desc}")
        neurons_fig.tight_layout()
        # neurons_fig.show()
        neurons_fig.savefig(f"{out_img_path}_layer-{layer_idx}-neurons.png")
        neurons_fig.clf()

    fig.suptitle(f"Extracted Layer Features - {desc}")
    fig.tight_layout()
    # fig.show()
    fig.savefig(f"{out_img_path}.png")
    fig.clf()

## Generate Some Runs
To infer what the model has learnt, we need to get the observations that are fed into the CNN

This doesn't need to be run if the images already exist

In [4]:
from os import listdir
from os.path import isfile, join
input_img_path = 'input_frame_stacks'
onlyfiles = [f for f in listdir(input_img_path) if isfile(join(input_img_path, f))] # get files in path
if not onlyfiles:
    print("Generating input frames")
    # Using A2C as it is able to get further
    env, env_name = create_env()

    # Apply wrappers to environment
    env = SkipFrame(env, skip=4)
    env = GrayScaleObservation(env, keep_dim=False) # Grayscale images
    env = ResizeObservation(env, shape=84) # image dim: [84, 84]
    env = FrameStack(env, num_stack=4) # 4 frames at a time
    obs = (4, 84, 84)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = ACNetwork(obs, env.action_space.n)
    checkpoint = torch.load('checkpoints/a2c/a2c_rollout10_ep120k.pt', map_location=device)
    model.load_state_dict(checkpoint['model'])
    model.cuda()
    model.eval()
    print(model)
    model_cnn = model.conv

    # run some steps
    def get_action(state):
        state = helper.normalize_states(helper.to_tensor(state)).to(device)
        action_probs = model.forward(state.unsqueeze(0))[0]
        return torch.distributions.Categorical(action_probs).sample().item()

    def get_obs(state):
        return np.array(state)

    run_model(env, 300, get_action, get_obs, input_img_path)
    env.close()
else:
    print("Input frames already exist, skipping generation...")

Input frames already exist, skipping generation...


## A2C


In [6]:
env, env_name = create_env()

# Apply wrappers to environment
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env, keep_dim=False) # Grayscale images
env = ResizeObservation(env, shape=84) # image dim: [84, 84]
env = FrameStack(env, num_stack=4) # 4 frames at a time
obs = (4, 84, 84)


for model_eps in ['120k']:#['100k', '120k']:
    model = ACNetwork(obs, env.action_space.n)
    model_path = f'checkpoints/a2c/a2c_rollout10_ep{model_eps}.pt'
    print(f"Loading A2C model from checkpoint: {model_path}")
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    print("Model Loaded, Ensuring weights setup for cuda")
    model.cuda()
    print("Evaluating model ready for CNN Feature Visualisation")
    model.eval()
    model_cnn = model.conv
    print(model_cnn)

    # Frames
    input_img_path = 'input_frame_stacks'
    frames = [
        ('frame_42', "Jumping Over Goomba"),
        ('frame_70', "Jumping Over Pipe"),
        ('frame_108',"Falling In-front Of Goomba (In Air)"),
        ('frame_111',"Falling In-front Of Goomba"),
        ('frame_230',"Stuck On Pipe"),
        ('frame_233',"Stuck On Pipe (In Air)")
    ]

    # visualise_cnn_layers
    n_step = 2048
    conv_layers = [('0', 32, 0), ('2', 32, 1), ('7', 64, 2)] # (layer_idx, features, conv_layer_idx)
    for file_id, desc in frames:
        if file_id != "NOISE":
            frame_path = os.path.join(input_img_path, f"{file_id}.png")
        else:
            frame_path = file_id
        visualise_cnn_layer_neurons(frame_path, model_cnn, conv_layers, device, n_step, f"CNN Extracted Features Visualisation\nA2C Model ({model_eps} Episodes) - Scenario: {desc}", f"out/A2C/{model_eps}ep_{n_step}-steps_{desc.replace(' ', '-')}")

env.close()

Loading A2C model from checkpoint: checkpoints/a2c/a2c_rollout10_ep120k.pt
Model Loaded, Ensuring weights setup for cuda
Evaluating model ready for CNN Feature Visualisation
Sequential(
  (0): Conv2d(4, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Processing Layer-0
Interrupted optimization at step 941.


## DQN

In [4]:
movement = [
    ['right'],
    ['right', 'A'],
    ['NOOP']
]
env, env_name = create_env(movement=movement)
env = SkipFrame(env, skip=4)
env = ResizeObservation(env, shape=84) # image dim: [84, 84]
env = GrayScaleObservation(env, keep_dim=False) # Grayscale images
env = FrameStack(env, num_stack=4) # 4 frames at a time
obs = (4, 84, 84)

for model_chpt in ['12']:#['8', '12']:
    model_path = f'checkpoints/dqn/mario_net_{model_chpt}.chkpt'
    print(f"Loading DQN model from checkpoint: {model_path}")
    model = Mario(state_dim=obs, action_dim=env.action_space.n, save_dir=".")
    path = Path(model_path)
    model.load(path)
    print("Model Loaded, Evaluating model ready for CNN Feature Visualisation")
    model.net.eval()
    model_cnn = model.net.online # online_features for new DQN algo (pending checkpoints)
    print(model_cnn)

    # Frames
    input_img_path = 'input_frame_stacks'
    frames = [
        ('frame_42', "Jumping Over Goomba"),
        ('frame_70', "Jumping Over Pipe"),
        ('frame_108',"Falling In-front Of Goomba (In Air)"),
        ('frame_111',"Falling In-front Of Goomba"),
        ('frame_230',"Stuck On Pipe"),
        ('frame_233',"Stuck On Pipe (In Air)")
    ]

    n_step=512
    # visualise_cnn_layers
    conv_layers = [('0', 32, 0), ('2', 32, 1), ('5', 64, 2), ('7', 64, 3)] # (layer_idx, features, conv_layer_idx)

    # # visualise_cnn_layers
    for file_id, desc in frames:
        if file_id != "NOISE":
            frame_path = os.path.join(input_img_path, f"{file_id}.png")
        else:
            frame_path = file_id
        visualise_cnn_layer_neurons(frame_path, model_cnn, conv_layers, device, n_step, f"CNN Extracted Features Visualisation\nDQN Model (Checkpoint {model_chpt}) - Scenario: {desc}", f"out/DQN/{model_chpt}chpt_{n_step}-steps_{desc.replace(' ', '-')}")

env.close()

Loading DQN model from checkpoint: checkpoints/dqn/mario_net_8.chkpt
Loading model at checkpoints/dqn/mario_net_8.chkpt with exploration rate 0.19691163516176424
Model Loaded, Evaluating model ready for CNN Feature Visualisation
Sequential(
  (0): Conv2d(4, 32, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (9): Flatten(start_dim=1, end_dim=-1)
  (10): Linear(in_features=1024, out_features=128, bias=True)
  (11): ReLU()
  (12): Linear(in_features=128, out_features=3, bias=True)
)
Processing Layer-0


/home/whiffingj/bin/anaconda3/envs/rl_vis/lib/python3.9/site-packages/lucent/optvis/render.py:103: UserWarning: Some layers could not be computed because the size of the image is not big enough. It is fine, as long as the noncomputed layers are not used in the objective function(exception details: 'mat1 and mat2 shapes cannot be multiplied (2x16384 and 1024x128)')
  warnings.warn(


Processing Layer-1
Processing Layer-2
Processing Layer-3
Processing Activation Image For Layer-0, 32
Processing Activation Image For Layer-1, 32
Processing Activation Image For Layer-2, 64
Processing Activation Image For Layer-3, 64


/tmp/ipykernel_20159/1798537024.py:134: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.tight_layout()


Processing Layer-0
Processing Layer-1
Processing Layer-2
Processing Layer-3
Processing Activation Image For Layer-0, 32
Processing Activation Image For Layer-1, 32
Processing Activation Image For Layer-2, 64
Processing Activation Image For Layer-3, 64
Processing Layer-0
Processing Layer-1
Processing Layer-2
Processing Layer-3
Processing Activation Image For Layer-0, 32


/tmp/ipykernel_20159/1798537024.py:119: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  neurons_fig = plt.figure(figsize=(18, 14))


Processing Activation Image For Layer-1, 32
Processing Activation Image For Layer-2, 64
Processing Activation Image For Layer-3, 64
Processing Layer-0
Processing Layer-1
Processing Layer-2
Processing Layer-3
Processing Activation Image For Layer-0, 32
Processing Activation Image For Layer-1, 32
Processing Activation Image For Layer-2, 64
Processing Activation Image For Layer-3, 64
Processing Layer-0
Processing Layer-1
Processing Layer-2
Processing Layer-3
Processing Activation Image For Layer-0, 32
Processing Activation Image For Layer-1, 32
Processing Activation Image For Layer-2, 64
Processing Activation Image For Layer-3, 64
Processing Layer-0
Processing Layer-1
Processing Layer-2
Processing Layer-3
Processing Activation Image For Layer-0, 32
Processing Activation Image For Layer-1, 32
Processing Activation Image For Layer-2, 64
Processing Activation Image For Layer-3, 64
Loading DQN model from checkpoint: checkpoints/dqn/mario_net_12.chkpt
Loading model at checkpoints/dqn/mario_net